In [8]:
# 📦 Step 1: Import and Set Credentials
import requests

# --- 🔐 Enter your Amadeus Sandbox credentials here ---
AMADEUS_CLIENT_ID = 'ID HERE'
AMADEUS_CLIENT_SECRET = 'ID HERE'


In [9]:
# 🔄 Step 2: Get Access Token
def get_amadeus_token(client_id, client_secret):
    response = requests.post(
        "https://test.api.amadeus.com/v1/security/oauth2/token",
        data={
            "grant_type": "client_credentials",
            "client_id": client_id,
            "client_secret": client_secret
        }
    )
    response.raise_for_status()
    return response.json()["access_token"]

token = get_amadeus_token(AMADEUS_CLIENT_ID, AMADEUS_CLIENT_SECRET)
print("✅ Token acquired")


✅ Token acquired


In [10]:
import pandas as pd
import statistics

# ✈️ Updated Flight Search
def search_flights(token, origin, destination, depart_date, return_date, adults=1, max_results=10):
    headers = {"Authorization": f"Bearer {token}"}
    params = {
        "originLocationCode": origin,
        "destinationLocationCode": destination,
        "departureDate": depart_date,
        "returnDate": return_date,
        "adults": adults,
        "max": max_results,
        "currencyCode": "USD"
    }
    response = requests.get(
        "https://test.api.amadeus.com/v2/shopping/flight-offers",
        headers=headers,
        params=params
    )
    response.raise_for_status()
    data = response.json().get("data", [])

    prices = []
    non_stop = []
    with_stops = []

    print(f"🛫 Found {len(data)} flight options:\n")

    for i, offer in enumerate(data, 1):
        airline = offer['validatingAirlineCodes'][0]
        price = float(offer['price']['total'])
        prices.append(price)

        itinerary = offer['itineraries'][0]
        segments = itinerary['segments']
        num_stops = len(segments) - 1
        duration = itinerary['duration'].replace("PT", "").lower()  # e.g., "PT7H10M" → "7h10m"

        if num_stops == 0:
            non_stop.append(price)
        else:
            with_stops.append(price)

        first_seg = segments[0]
        last_seg = segments[-1]

        dep_code = first_seg['departure']['iataCode']
        dep_time = first_seg['departure']['at']
        arr_code = last_seg['arrival']['iataCode']
        arr_time = last_seg['arrival']['at']

        print(f"{i}. {airline} | {dep_code} {dep_time} → {arr_code} {arr_time} | Stops: {num_stops} | Duration: {duration} | Price: ${price:.2f}")


    if prices:
        cheapest = min(prices)
        median_price = statistics.median(prices)
        print(f"\n💰 Cheapest flight: ${cheapest:.2f}")
        print(f"💰 Median flight price: ${median_price:.2f}")

        if non_stop:
            cheapest_non_stop = min(non_stop)
            print(f"✅ Cheapest Non-stop: ${cheapest_non_stop:.2f}")
        if with_stops:
            cheapest_stops = min(with_stops)
            print(f"🔁 Cheapest With Stops: ${cheapest_stops:.2f}")
    else:
        print("No flights found.")

    return prices

In [11]:
# 🏨 Updated Hotel Search
# 🏨 Helper: Get hotel IDs for a city
def get_hotel_ids_by_city(token, city_code, max_hotels=20):
    headers = {"Authorization": f"Bearer {token}"}
    params = {
        "cityCode": city_code,
        "radius": 10,  # km radius from city center
        "radiusUnit": "KM",
        "hotelSource": "ALL"
    }
    response = requests.get(
        "https://test.api.amadeus.com/v1/reference-data/locations/hotels/by-city",
        headers=headers,
        params=params
    )
    response.raise_for_status()
    hotels = response.json().get("data", [])

    hotel_ids = [hotel["hotelId"] for hotel in hotels][:max_hotels]
    print(f"🏨 Found {len(hotel_ids)} hotel IDs for city {city_code}")
    return hotel_ids

    hotel_list = []
    for h in hotels:
        name = h["hotel"]["name"]
        address = h["hotel"].get("address", {}).get("lines", ["N/A"])[0]
        price = float(h["offers"][0]["price"]["total"])
        currency = h["offers"][0]["price"]["currency"]

        hotel_list.append({
            "Hotel Name": name,
            "Address": address,
            "Price": price,
            "Currency": currency
        })

    df = pd.DataFrame(hotel_list)
    if df.empty:
        print("No hotels found.")
        return df

    df_sorted = df.sort_values("Price").reset_index(drop=True)
    cheapest = df_sorted.iloc[0]["Price"]
    median_price = df_sorted["Price"].median()

    print(df_sorted)
    print(f"\n🏨 Cheapest Hotel: ${cheapest:.2f}")
    print(f"🏨 Median Hotel Price: ${median_price:.2f}")

    return df_sorted

In [12]:
# 🚀 Step 5: Use the Functions with Example Inputs
origin = "JFK"
destination = "LHR"
depart_date = "2025-07-15"
return_date = "2025-07-22"
city_code = destination
check_in = depart_date
check_out = return_date

hotel_ids = get_hotel_ids_by_city(token, city_code)
search_hotels_by_id(token, hotel_ids, check_in, check_out)

# Run the searches
search_flights(token, origin, destination, depart_date, return_date)
print("\n" + "-"*50 + "\n")


🏨 Found 20 hotel IDs for city LHR
                               Hotel Name Address    Price Currency
0                Heathrow Airport (M4/J4)     N/A   350.00      GBP
1         Novotel London Heathrow Airport     N/A   473.33      GBP
2  Sheraton Skyline Hotel London Heathrow     N/A   669.75      GBP
3                 Sheraton Heathrow Hotel     N/A   797.25      GBP
4                  HILTON HEATHROW TERM 5     N/A   973.00      GBP
5            Heathrow Airport (Bath Road)     N/A  6993.00      GBP
6             Heathrow Airport Terminal 5     N/A  6993.00      GBP

🏨 Cheapest Hotel: $350.00
🏨 Median Hotel Price: $797.25
🛫 Found 10 flight options:

1. B6 | JFK 2025-07-15T20:59:00 → LHR 2025-07-16T09:30:00 | Stops: 0 | Duration: 7h31m | Price: $557.41
2. VS | JFK 2025-07-15T23:00:00 → LHR 2025-07-16T11:10:00 | Stops: 0 | Duration: 7h10m | Price: $567.41
3. VS | JFK 2025-07-15T23:00:00 → LHR 2025-07-16T11:10:00 | Stops: 0 | Duration: 7h10m | Price: $567.41
4. VS | JFK 2025-07-15T19